In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder



# Read JSON file

In [2]:
with open('data.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

# Label Encoder

In [3]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

# Tokenization

In [4]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# Create a train the chatbot 

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))  # Remove input_length parameter
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - accuracy: 0.3674 - loss: 1.9581
Epoch 207/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step - accuracy: 0.3570 - loss: 1.6857
Epoch 208/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.9511
Epoch 209/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.9542
Epoch 210/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.9805
Epoch 211/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.9141
Epoch 212/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.3674 - loss: 1.7500
Epoch 213/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.8582
Epoch 214/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step - accuracy: 0.3570 - loss: 1.6439
Epoch 215/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - accuracy: 0.3674 - loss: 1.7429
Epoch 216/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.8934
Epoch 217/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3570

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3570 - loss: 1.4878
Epoch 275/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.6043
Epoch 276/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.5764
Epoch 277/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.7671
Epoch 278/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.7267
Epoch 279/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.7007
Epoch 280/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3570 - loss: 1.3748
Epoch 281/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.5533
Epoch 282/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3570 - loss: 1.3267 
Epoch 283/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3674 - loss: 1.8221
Epoch 284/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4795 - loss: 1.4767
Epoch 285/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5101 - loss: 

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - accuracy: 0.3980 - loss: 1.6261
Epoch 343/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3980 - loss: 1.5552
Epoch 344/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3980 - loss: 1.6271
Epoch 345/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4182 - loss: 1.4127
Epoch 346/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5511 - loss: 1.5575
Epoch 347/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step - accuracy: 0.5407 - loss: 1.3713
Epoch 348/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5511 - loss: 1.5626
Epoch 349/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5407 - loss: 1.1751
Epoch 350/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 941us/step - accuracy: 0.5818 - loss: 1.5422
Epoch 351/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5713 - loss: 1.3835
Epoch 352/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - accuracy: 0.5818 - loss: 1.5159
Epoch 353/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4795 -

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6020 - loss: 0.8577
Epoch 411/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.6020 - loss: 1.1927
Epoch 412/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6124 - loss: 1.3624
Epoch 413/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6020 - loss: 1.1657
Epoch 414/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6020 - loss: 1.0021
Epoch 415/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - accuracy: 0.5818 - loss: 1.3324
Epoch 416/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5818 - loss: 1.3884
Epoch 417/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - accuracy: 0.6020 - loss: 1.1359
Epoch 418/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6020 - loss: 0.9318
Epoch 419/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - accuracy: 0.6020 - loss: 1.0459
Epoch 420/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6020 - loss: 0.8138
Epoch 421/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step - accuracy: 0.6124

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - accuracy: 0.6632 - loss: 0.8281
Epoch 479/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6632 - loss: 0.7707
Epoch 480/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6736 - loss: 1.1214
Epoch 481/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6632 - loss: 0.6693
Epoch 482/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6736 - loss: 1.1656
Epoch 483/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6632 - loss: 0.8012
Epoch 484/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6632 - loss: 0.6926
Epoch 485/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6632 - loss: 0.9755
Epoch 486/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6632 - loss: 0.6633
Epoch 487/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step - accuracy: 0.6632 - loss: 0.7863
Epoch 488/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6632 - loss: 1.0606
Epoch 489/500
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6632 - los

# Saving the neural network

In [6]:
# Save the trained model in native Keras format
model.save("chat_model.keras")

import pickle

# Save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# Save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)


# Testing the Chatbot

In [ ]:
import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("data.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model.keras')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: Who are you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
ChatBot: I'm Joana, an Artificial Intelligent bot
User: what do you do?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
ChatBot: Tell me your problem to assist you
User: I dont know how to create an account
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/ste ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
ChatBot: Just go to our web site and follow the guidelines to create a new account
User: 